In [ ]:
# 确定当前路径, 方便指定词典路径
import os
!cd

In [ ]:
import sys
import datetime
import pandas as pd

def callbackMsg(log):
    print(log)
    
def prepareData(input_table, cbFunc):
    df = pd.DataFrame(input_table, columns=['生产厂家名称'])
    df.drop_duplicates(inplace=True)

    df['shougong'] = df['生产厂家名称']
    
    ### 替换（集团）为集团
    df.replace({'shougong':'\(集团\)|（集团）'},'集团',regex=True, inplace=True)
    
    ### 去所有(*)  
    df.replace({'shougong':'\([\u4e00-\u9fa5\w&]{1,}\)'},'',regex=True, inplace=True)
    
    ###shougong提取总部   
    df_temp = df['shougong'].str.extract(r'(?P<org0>[\u4e00-\u9fa5\w\(\)&]{2,}(?:集团|公司|研究所|研究院|电业局|总公司|总厂|厂|所|大学){1,})(?P<org1>[\u4e00-\u9fa5\w\(\)&]*(?:集团|分公司|公司|研究所|研究院|总厂|厂){0,})(?P<org2>[\u4e00-\u9fa5\w\(\)&]*(?:集团|分公司|公司|分厂|厂|研究所|研究院|中心){1,})[\u4e00-\u9fa5\w\(\)&]*', expand=True)

    df = pd.concat([df,df_temp],axis=1)
    df.loc[df['org0'].str.len()>0,'shougong']=df['org0']
    df['temp'] = df['shougong'].str.extract(r'(?P<org>[\u4e00-\u9fa5\w\(\)&]*(?:集团|公司|厂|研究所|研究院|株式会社|经销部|经营部|电业局|分院|学院|中心|大学){1,}$)',expand=False)
    df.drop(labels=['shougong','org0','org1','org2'],axis=1, inplace=True)

    output_table_1 = df[df['temp'].isnull().values == True]
    df = df[df['temp'].isnull().values != True]
    log = sys._getframe().f_code.co_name + "=>列名：[" + ','.join(df.columns.values.tolist()) + "], 记录数：%5d" % (df.shape[0])
    cbFunc(log)

    dflist = [output_table_1, df]  
    return dflist

In [ ]:
DATA_SRC_DIR='..\workspace\input-data'
DATA_OUT_DIR='..\workspace\output-data'

inFile = os.path.join(DATA_SRC_DIR,"设备供应商名单-demo.xlsx")

df = pd.DataFrame(pd.read_excel(inFile,encoding='utf-8'))

dflist = prepareData(df,callbackMsg)

df = dflist[1]
print(df)

outFile = os.path.join(DATA_OUT_DIR,"设备供应商名单-demo-预处理.xlsx")
writer = pd.ExcelWriter(outFile,encoding='utf-8')
df.to_excel(writer,'设备供应商')
writer.save()